# Prediksi Gagal Jantung

## Import Data

In [1]:
# Import library pengolahan struktur data
import pandas as pd

# Import library pengolahan angka
import numpy as np

In [2]:
## Bikin Fungsinya
def import_data(filename):
    """
    Fungsi import data dan hapus duplikat
    :param filename: string nama file.csv
    :return df: <pandas data frame> sampel data
    """
    
    # print nama file
    print("Source data : ",filename)
    
    # read data
    df = pd.read_csv(filename)
    print("Data asli : ",df.shape,"(observasi, kolom)")
    
    # hapus duplikat
    df = df.drop_duplicates()
    print("Data setelah drop dulikat :",df.shape,"(observasi, kolom)")
    
    #jangan lupa return
    return df
    
# state lokasi file
data_src = "/home/jovyan/work/project/heart_failure_clinical_records_dataset.csv"

# jalankan import data
heartfail_df = import_data(filename = data_src)

Source data :  /home/jovyan/work/project/heart_failure_clinical_records_dataset.csv
Data asli :  (299, 13) (observasi, kolom)
Data setelah drop dulikat : (299, 13) (observasi, kolom)


In [3]:
heartfail_df.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


## Data Preprocessing

### Input-output Split

In [4]:
# fungsi ekstrak
def extract_input_output(data, 
                         output_column_name):
    """
    Fungsi untuk memisahkan data input dan output, dengan hanya memasukkan data master dan kolom output
    :param data: <pandas data frame> data seluruh sample
    :param output_column_name: <string> nama kolom output
    :return input_data: <pandas dataframe> data input
    :return output_data: <pandas dataframe> data output
    """
    
    output_data = data[output_column_name]
    input_data = data.drop(output_column_name,
                           axis = 1)
    
    return input_data, output_data

x, y = extract_input_output(data = heartfail_df,
                            output_column_name = 'DEATH_EVENT')

### Train-test split

In [5]:
# Import train-test splitting library dari sklearn (scikit learn)
from sklearn.model_selection import train_test_split

# Train test split
x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    test_size = 0.25,
                                                    random_state = 12)

In [6]:
# Sanity check hasil splitting
print(x_train.shape)
print(x_test.shape)

(224, 12)
(75, 12)


In [7]:
# Ratio
x_test.shape[0] / x.shape[0]

0.2508361204013378

In [8]:
x_train.isnull().any()

age                         False
anaemia                     False
creatinine_phosphokinase    False
diabetes                    False
ejection_fraction           False
high_blood_pressure         False
platelets                   False
serum_creatinine            False
serum_sodium                False
sex                         False
smoking                     False
time                        False
dtype: bool

### Cek heatmap

In [9]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# plt.figure(figsize=(10,10))
# sns.heatmap(heartfail_df.corr(), vmin=-1, cmap='coolwarm', annot=True);

### Standarizer Variable

In [10]:
from sklearn.preprocessing import StandardScaler

# Buat fungsi
def standardizerData(data):
    """
    Fungsi untuk melakukan standarisasi data
    :param data: <pandas dataframe> sampel data
    :return standardized_data: <pandas dataframe> sampel data standard
    :return standardizer: method untuk standardisasi data
    """
    data_columns = data.columns  # agar nama kolom tidak hilang
    data_index = data.index  # agar index tidak hilang

    # buat (fit) standardizer
    standardizer = StandardScaler()
    standardizer.fit(data)

    # transform data
    standardized_data_raw = standardizer.transform(data)
    standardized_data = pd.DataFrame(standardized_data_raw)
    standardized_data.columns = data_columns
    standardized_data.index = data_index

    return standardized_data, standardizer

In [11]:
x_train_clean, standardizer = standardizerData(data = x_train)

In [12]:
x_train_clean.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time
205,-0.943131,1.123666,-0.446449,1.123666,0.653262,-0.723747,0.975605,-0.397082,-0.108976,-1.328696,-0.695301,0.732795
233,-0.690258,1.123666,0.123552,-0.889944,0.041731,-0.723747,0.658760,0.033964,0.119160,0.752618,1.438226,1.020198
38,-0.100221,-0.889944,2.180832,1.123666,-0.657162,-0.723747,0.411225,1.003818,0.119160,0.752618,-0.695301,-1.318215
24,1.164144,-0.889944,-0.008393,1.123666,-0.657162,1.381699,-0.001089,0.497339,-0.565247,-1.328696,-0.695301,-1.409661
178,0.152652,1.123666,-0.493949,1.123666,1.963687,-0.723747,0.034972,-0.181559,1.944247,0.752618,-0.695301,0.210245


## Training Model

In [13]:
y_train.value_counts(normalize = True)

DEATH_EVENT
0    0.6875
1    0.3125
Name: proportion, dtype: float64

In [14]:
# Import dari sklearn
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

In [15]:
# Model K nearest neighbor
knn = KNeighborsClassifier(
    n_neighbors=7, 
    weights='uniform', 
    metric='manhattan'
)
knn.fit(x_train_clean, y_train)

# from sklearn.model_selection import train_test_split, GridSearchCV

# knn = KNeighborsClassifier()
# param_grid = {'n_neighbors': [1, 2, 3, 5, 7, 9], 'weights': ['uniform', 'distance'],'metric': ['euclidean', 'manhattan']}

# # Buat objek GridSearchCV
# grid_search = GridSearchCV(knn, param_grid, cv=5)

# # Lakukan grid search pada data train
# grid_search.fit(x_train_clean, y_train)

# # Cetak parameter terbaik dan skor akurasi terbaik
# print('Parameter terbaik:', grid_search.best_params_)
# print('Skor akurasi terbaik:', grid_search.best_score_)

KNeighborsClassifier(metric='manhattan', n_neighbors=7)

In [16]:
# # Model Logistic Regression
logreg = LogisticRegression(
    C=0.01, 
    penalty='l2', 
    solver='liblinear'
)
logreg.fit(x_train_clean, y_train)

# from sklearn.model_selection import train_test_split, GridSearchCV

# param_grid = {
#     'penalty': ['l1', 'l2'],
#     'C': [0.01, 0.1, 1, 10, 100],
#     'solver': ['liblinear', 'saga']
# }

# lr = LogisticRegression(random_state=42)

# # Buat objek GridSearchCV
# grid_search = GridSearchCV(lr, param_grid, cv=5)

# # Lakukan grid search pada data train
# grid_search.fit(x_train_clean, y_train)

# # Cetak parameter terbaik dan skor akurasi terbaik
# print('Parameter terbaik:', grid_search.best_params_)
# print('Skor akurasi terbaik:', grid_search.best_score_)

LogisticRegression(C=0.01, solver='liblinear')

In [17]:
# Model Random Forest Classifier
random_forest = RandomForestClassifier(
    max_depth=10, 
    max_features='sqrt', 
    min_samples_leaf=1, 
    min_samples_split=10, 
    n_estimators=50
)
random_forest.fit(x_train_clean, y_train)

RandomForestClassifier(max_depth=10, min_samples_split=10, n_estimators=50)

In [18]:
# Model Support Vector Classifier
svm = SVC(
    kernel='linear', 
    C=10, 
    gamma='scale'
)
svm.fit(x_train_clean, y_train)

# from sklearn.model_selection import train_test_split, GridSearchCV

# param_grid = {
#     'C': [0.1, 1, 10, 100],
#     'kernel': ['linear', 'rbf', 'poly'],
#     'gamma': ['scale', 'auto']
# }

# svm = SVC(random_state=42)

# # Buat objek GridSearchCV
# grid_search = GridSearchCV(svm, param_grid, cv=5)

# # Lakukan grid search pada data train
# grid_search.fit(x_train_clean, y_train)

# # Cetak parameter terbaik dan skor akurasi terbaik
# print('Parameter terbaik:', grid_search.best_params_)
# print('Skor akurasi terbaik:', grid_search.best_score_)

SVC(C=10, kernel='linear')

In [19]:
# Model Naive Bayes
naive_bayes = GaussianNB(var_smoothing=1e-09)
naive_bayes.fit(x_train_clean, y_train)

# from sklearn.model_selection import train_test_split, GridSearchCV

# param_grid = {
#     'var_smoothing': [1e-09, 1e-08, 1e-07, 1e-06]
# }

# nb = GaussianNB()

# # Buat objek GridSearchCV
# grid_search = GridSearchCV(nb, param_grid, cv=5)

# # Lakukan grid search pada data train
# grid_search.fit(x_train_clean, y_train)

# # Cetak parameter terbaik dan skor akurasi terbaik
# print('Parameter terbaik:', grid_search.best_params_)
# print('Skor akurasi terbaik:', grid_search.best_score_)

GaussianNB()

In [20]:
# Model decision tree
dtc = DecisionTreeClassifier(
    max_depth=3, 
    min_samples_leaf=2, 
    min_samples_split=5
)
dtc.fit(x_train_clean, y_train)

# # Buat model Decision Tree
# dt = DecisionTreeClassifier()

# # Tentukan nilai parameter yang akan diuji
# param_grid = {'max_depth': [3, 5, 7], 
#               'min_samples_split': [2, 5, 10], 
#               'min_samples_leaf': [1, 2, 4]}

# # Buat objek GridSearchCV
# grid_search = GridSearchCV(dt, param_grid, cv=5)

# # Lakukan grid search pada data train
# grid_search.fit(x_train_clean, y_train)

# # Cetak parameter terbaik dan skor akurasi terbaik
# print('Parameter terbaik:', grid_search.best_params_)
# print('Skor akurasi terbaik:', grid_search.best_score_)

DecisionTreeClassifier(max_depth=3, min_samples_leaf=2, min_samples_split=5)

### Prediction

In [21]:
predicted_logreg = pd.DataFrame(logreg.predict(x_train_clean))
predicted_logreg.head()

,0
0,0
1,0
2,1
3,1
4,0


In [22]:
predicted_knn = pd.DataFrame(knn.predict(x_train_clean))
predicted_knn.head()

,0
0,0
1,0
2,0
3,1
4,0


In [23]:
predicted_rf = pd.DataFrame(random_forest.predict(x_train_clean))
predicted_rf.head()

,0
0,0
1,0
2,1
3,1
4,0


In [24]:
predicted_svm = pd.DataFrame(svm.predict(x_train_clean))
predicted_svm.head()

,0
0,0
1,0
2,1
3,1
4,0


In [25]:
predicted_naive_bayes = pd.DataFrame(naive_bayes.predict(x_train_clean))
predicted_naive_bayes.head()

,0
0,0
1,0
2,1
3,1
4,0


In [26]:
predicted_dtc = pd.DataFrame(dtc.predict(x_train_clean))
predicted_dtc.head()

,0
0,0
1,0
2,1
3,1
4,0


In [27]:
benchmark = y_train.value_counts(normalize=True)
benchmark

DEATH_EVENT
0    0.6875
1    0.3125
Name: proportion, dtype: float64

In [28]:
# akurasi knn
knn.score(x_train_clean, y_train)

0.7946428571428571

In [29]:
# akurasi logistic regression
logreg.score(x_train_clean, y_train)

0.8526785714285714

In [30]:
# akurasi random forest
random_forest.score(x_train_clean, y_train)

0.9241071428571429

In [31]:
# akurasi Support Vector Classifier
svm.score(x_train_clean, y_train)

0.8482142857142857

In [32]:
# akurasi Naive Bayes
naive_bayes.score(x_train_clean, y_train)

0.78125

In [33]:
# akurasi Decision Tree
dtc.score(x_train_clean, y_train)

0.8526785714285714

### Cross Validation Score

In [34]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification

# Membuat objek k-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=1)

# Melakukan cross-validation pada model
results_logreg = cross_val_score(logreg, x_train_clean, y_train, cv=kfold)
results_knn = cross_val_score(knn, x_train_clean, y_train, cv=kfold)
results_rf = cross_val_score(random_forest, x_train_clean, y_train, cv=kfold)
results_svm = cross_val_score(svm, x_train_clean, y_train, cv=kfold)
results_nb = cross_val_score(naive_bayes, x_train_clean, y_train, cv=kfold)
results_dtc = cross_val_score(dtc, x_train_clean, y_train, cv=kfold)

# Menampilkan hasil performa model
print("Accuracy Log reg: %.2f%% (%.2f%%)" % (results_logreg.mean()*100, results_logreg.std()*100))
print("Accuracy KNN: %.2f%% (%.2f%%)" % (results_knn.mean()*100, results_knn.std()*100))
print("Accuracy RF: %.2f%% (%.2f%%)" % (results_rf.mean()*100, results_rf.std()*100))
print("Accuracy SVM: %.2f%% (%.2f%%)" % (results_svm.mean()*100, results_svm.std()*100))
print("Accuracy Naive Bayes: %.2f%% (%.2f%%)" % (results_nb.mean()*100, results_nb.std()*100))
print("Accuracy Decision Tree: %.2f%% (%.2f%%)" % (results_dtc.mean()*100, results_dtc.std()*100))

Accuracy Log reg: 79.89% (4.77%)
Accuracy KNN: 72.76% (3.89%)
Accuracy RF: 83.46% (2.41%)
Accuracy SVM: 79.42% (6.22%)
Accuracy Naive Bayes: 76.77% (3.16%)
Accuracy Decision Tree: 79.90% (6.32%)


### Save ke Pickle

In [35]:
import joblib

# Simpan model logreg ke dalam folder yang sama dengan notebook
# dengan nama logreg.pkl
joblib.dump(logreg, "./pickle/logreg.pkl")
joblib.dump(knn, "./pickle/knn.pkl")
joblib.dump(random_forest, "./pickle/random_forest.pkl")
joblib.dump(svm, "./pickle/svm.pkl")
joblib.dump(naive_bayes, "./pickle/naive_bayes.pkl")
joblib.dump(dtc, "./pickle/decision_tree.pkl")
#joblib.dump(random_forest_1, "random_forest_1.pkl")

['./pickle/decision_tree.pkl']

In [36]:
def extract_test(data, standardizer):
    cleaned_data = pd.DataFrame(standardizer.transform(data))
    cleaned_data.columns = data.columns
    
    return cleaned_data

In [37]:
def testPrediction(x_test, y_test, classifier, compute_score):
    """
    Fungsi untuk mendapatkan prediksi dari model
    :param X_test: <pandas dataframe> input
    :param y_test: <pandas series> output/target
    :param classifier: <sklearn method> model klasifikasi
    :param compute_score: <bool> True: menampilkan score, False: tidak
    :return test_predict: <list> hasil prediksi data input
    :return score: <float> akurasi model
    """
    if compute_score:
        score = classifier.score(x_test, y_test)
        print(f"Accuracy : {score:.4f}")

    test_predict = classifier.predict(x_test)

    return test_predict, score

In [38]:
x_test_clean = extract_test(data = x_test,
                           standardizer = standardizer)

In [39]:
x_test_clean.shape

(75, 12)

In [40]:
x_test_clean

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time
0,-1.617459,1.123666,-0.531949,-0.889944,-0.220354,-0.723747,1.005309,-0.289321,0.575432,0.752618,1.438226,0.915688
1,1.164144,-0.889944,-0.008393,-0.889944,0.216454,-0.723747,-0.001089,-0.203111,0.119160,0.752618,-0.695301,-0.312306
2,0.911271,1.123666,-0.506615,-0.889944,-1.093971,-0.723747,0.104282,-0.397082,0.803568,0.752618,1.438226,-0.860983
3,-0.184512,1.123666,-0.486560,-0.889944,0.653262,1.381699,0.975605,-0.289321,0.575432,0.752618,1.438226,-0.129413
4,-0.943131,-0.889944,-0.008393,-0.889944,2.138410,1.381699,-1.153196,-0.612605,0.803568,0.752618,1.438226,0.798114
...,...,...,...,...,...,...,...,...,...,...,...,...
70,-0.690258,-0.889944,-0.415838,-0.889944,1.963687,-0.723747,-0.430394,-0.720367,-0.793383,0.752618,1.438226,0.040416
71,0.068361,1.123666,0.068663,-0.889944,0.216454,-0.723747,0.193394,-0.720367,-0.793383,-1.328696,-0.695301,1.333728
72,1.164144,-0.889944,-0.497115,-0.889944,1.090070,1.381699,-0.153155,-0.289321,2.628654,0.752618,-0.695301,1.020198
73,0.911271,1.123666,0.372663,-0.889944,-1.093971,1.381699,0.737971,0.357249,0.575432,0.752618,1.438226,-0.260050


In [41]:
# Logistic Regression Performance
logreg_test_predict, score = testPrediction(x_test = x_test_clean,
                                            y_test = y_test,
                                            classifier = logreg,
                                            compute_score = True)

Accuracy : 0.8267


In [42]:
# K nearest neighbor Performance
knn_test_predict, score = testPrediction(x_test = x_test_clean,
                                        y_test = y_test,
                                        classifier = knn,
                                        compute_score = True)

Accuracy : 0.7333


In [43]:
# Random Forest Performance
rf_test_predict, score = testPrediction(x_test = x_test_clean,
                                        y_test = y_test,
                                        classifier = random_forest,
                                        compute_score = True)

Accuracy : 0.8533


In [44]:
# SVM
svm_test_predict, score = testPrediction(x_test = x_test_clean,
                                        y_test = y_test,
                                        classifier = svm,
                                        compute_score = True)

Accuracy : 0.8000


In [45]:
# Naive Bayes
nb_test_predict, score = testPrediction(x_test = x_test_clean,
                                        y_test = y_test,
                                        classifier = naive_bayes,
                                        compute_score = True)

Accuracy : 0.7600


In [46]:
# Decision Tree
dtc_test_predict, score = testPrediction(x_test = x_test_clean,
                                        y_test = y_test,
                                        classifier = dtc,
                                        compute_score = True)

Accuracy : 0.8267


In [47]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification

# Membuat objek k-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=1)

# Melakukan cross-validation pada model
results_logreg = cross_val_score(logreg, x_test_clean, y_test, cv=kfold)
results_knn = cross_val_score(knn, x_test_clean, y_test, cv=kfold)
results_rf = cross_val_score(random_forest, x_test_clean, y_test, cv=kfold)
results_svm = cross_val_score(svm, x_test_clean, y_test, cv=kfold)
results_nb = cross_val_score(naive_bayes, x_test_clean, y_test, cv=kfold)
results_dtc = cross_val_score(dtc, x_test_clean, y_test, cv=kfold)

# Menampilkan hasil performa model
print("Accuracy Log reg: %.2f%% (%.2f%%)" % (results_logreg.mean()*100, results_logreg.std()*100))
print("Accuracy KNN: %.2f%% (%.2f%%)" % (results_knn.mean()*100, results_knn.std()*100))
print("Accuracy RF: %.2f%% (%.2f%%)" % (results_rf.mean()*100, results_rf.std()*100))
print("Accuracy SVM: %.2f%% (%.2f%%)" % (results_svm.mean()*100, results_svm.std()*100))
print("Accuracy Naive Bayes: %.2f%% (%.2f%%)" % (results_nb.mean()*100, results_nb.std()*100))
print("Accuracy Decision Tree: %.2f%% (%.2f%%)" % (results_dtc.mean()*100, results_dtc.std()*100))

Accuracy Log reg: 86.67% (4.22%)
Accuracy KNN: 70.67% (6.80%)
Accuracy RF: 88.00% (6.53%)
Accuracy SVM: 86.67% (5.96%)
Accuracy Naive Bayes: 76.00% (11.62%)
Accuracy Decision Tree: 82.67% (9.98%)
